In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append(r"C:\Users\Aditya Gautam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages")

import yfinance as yf

import matplotlib.pyplot as plt
from ta.volatility import AverageTrueRange
from ta.momentum import RSIIndicator
from ta.trend import MACD, ADXIndicator
from pykalman import KalmanFilter

# Step 1: Fetch Bitcoin Data
def fetch_data():
    print("Fetching Bitcoin data...")
    data = yf.download("BTC-USD", start="2021-01-01", end="2024-12-31")
    data['Returns'] = data['Adj Close'].pct_change() * 100
    print("Data fetched successfully!")
    return data

# Step 2: Add Heikin-Ashi Candles
def add_heikin_ashi(data):
    print("Calculating Heikin-Ashi candles...")
    data['HA_Close'] = (data['Open'] + data['High'] + data['Low'] + data['Close']) / 4
    data['HA_Open'] = (data['Open'].shift(1) + data['Close'].shift(1)) / 2
    data['HA_High'] = data[['High', 'HA_Open', 'HA_Close']].max(axis=1)
    data['HA_Low'] = data[['Low', 'HA_Open', 'HA_Close']].min(axis=1)
    print("Heikin-Ashi candles added successfully!")
    return data

# Step 3: Add Technical Indicators
def add_indicators(data):
    print("Adding technical indicators...")
    
    # ATR
    atr = AverageTrueRange(high=data['High'], low=data['Low'], close=data['Close'], window=14)
    data['ATR'] = atr.average_true_range()
    
    # RSI
    rsi = RSIIndicator(close=data['Close'], window=14)
    data['RSI'] = rsi.rsi()
    
    # MACD
    macd = MACD(close=data['Close'], window_slow=26, window_fast=12, window_sign=9)
    data['MACD'] = macd.macd()
    data['Signal_Line'] = macd.macd_signal()
    data['MACD_Hist'] = macd.macd_diff()
    
    # ADX
    adx = ADXIndicator(high=data['High'], low=data['Low'], close=data['Close'], window=14)
    data['ADX'] = adx.adx()
    data['+DI'] = adx.adx_pos()
    data['-DI'] = adx.adx_neg()
    
    print("Indicators added successfully!")
    return data

# Step 4: Apply Kalman Filter for Smoothing
def apply_kalman(data):
    print("Applying Kalman Filter for smoothing...")
    kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)
    state_means, _ = kf.em(data['Close'].dropna().values, n_iter=10).filter(data['Close'].dropna().values)
    data['Close_Kalman'] = pd.Series(state_means, index=data['Close'].dropna().index)
    print("Kalman Filter applied successfully!")
    return data

# Step 5: Add Fibonacci Levels
def add_fibonacci(data):
    print("Calculating Fibonacci retracement levels...")
    max_price = data['High'].max()
    min_price = data['Low'].min()
    diff = max_price - min_price
    data['Fib_61.8'] = max_price - diff * 0.618
    data['Fib_38.2'] = max_price - diff * 0.382
    print("Fibonacci levels added successfully!")
    return data

# Step 6: Generate Trading Signals
def generate_signals(data):
    print("Generating trading signals...")
    
    # Long Signal
    data['Long_Signal'] = (
        (data['RSI'] < 30) & 
        (data['MACD'] > data['Signal_Line']) & 
        (data['ATR'] > 1.5) &
        (data['Close'] > data['Fib_61.8'])  # Near Fibonacci support
    )
    
    # Short Signal
    data['Short_Signal'] = (
        (data['RSI'] > 70) & 
        (data['MACD'] < data['Signal_Line']) & 
        (data['ATR'] > 1.5) &
        (data['Close'] < data['Fib_38.2'])  # Near Fibonacci resistance
    )
    
    print("Signals generated successfully!")
    return data

# Step 7: Backtest Strategy
def backtest(data):
    print("Backtesting the strategy...")
    
    # Initialize position and strategy returns
    data['Position'] = 0
    data.loc[data['Long_Signal'], 'Position'] = 1
    data.loc[data['Short_Signal'], 'Position'] = -1
    data['Strategy_Returns'] = data['Position'].shift(1) * data['Returns'] / 100
    data['Cumulative_Strategy'] = (1 + data['Strategy_Returns']).cumprod()
    data['Cumulative_Market'] = (1 + data['Returns'] / 100).cumprod()
    
    # Plot Results
    plt.figure(figsize=(12, 6))
    plt.plot(data.index, data['Cumulative_Strategy'], label='Strategy Performance', color='green')
    plt.plot(data.index, data['Cumulative_Market'], label='Market Performance', color='blue')
    plt.title('Backtest Results: Strategy vs Market')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Returns')
    plt.legend()
    plt.show()
    
    print("Backtesting completed successfully!")
    return data

# Main Program Execution
def main():
    print("Starting Bitcoin analysis and prediction...")
    
    # Step 1: Fetch Data
    bitcoin_data = fetch_data()
    
    # Step 2: Add Heikin-Ashi Candles
    bitcoin_data = add_heikin_ashi(bitcoin_data)
    
    # Step 3: Add Indicators
    bitcoin_data = add_indicators(bitcoin_data)
    
    # Step 4: Apply Kalman Filter
    bitcoin_data = apply_kalman(bitcoin_data)
    
    # Step 5: Add Fibonacci Levels
    bitcoin_data = add_fibonacci(bitcoin_data)
    
    # Step 6: Generate Trading Signals
    bitcoin_data = generate_signals(bitcoin_data)
    
    # Step 7: Backtest Strategy
    backtest(bitcoin_data)
    
    print("Analysis and prediction completed!")

# Run the Program
if _name_ == "_main_":
    main()

ModuleNotFoundError: No module named 'pykalman'